In [ ]:
import numpy as np
import pandas as pd
import io
import requests
from statsmodels.formula.api import logit
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('seaborn-v0_8-whitegrid')

# SHED
## Survey of Household Economics and Decisionmaking
### Board of Governors of the Federal Reserve Board
The report draws from the Board's 11th annual Survey of Household Economics and Decisionmaking, (SHED), which examines the financial lives of U.S. adults and their families. The survey of more than 11,000 adults was conducted in October 2023. The report discusses findings related to financial well-being, income, employment, expenses, banking and credit, housing, higher education and student loans, and retirement and investments.

https://www.federalreserve.gov/consumerscommunities/shed_data.htm

In [ ]:
site = r'https://www.federalreserve.gov/consumerscommunities/files/SHED_public_use_data_2023_(CSV).zip'
s = requests.get(site).content
shed = pd.read_csv(io.BytesIO(s), compression='zip', header=0, low_memory=False) 


In [ ]:
shed.head()

In [ ]:
shed.shape

In [ ]:
shed.columns

In [ ]:
work_col_map = {'D1A': 'work_for_pay', 'D4': 'more_than_one_job', 'D48':'work_more_35hrs_all_jobs',
               'D3B':'work_more_35hrs_main_job', 'D1E': 'want_to_work_more', 
               'D3A': 'work_for_others_or_self_employed', 'D44_f': 'lost_job',
                'D44_a': 'asked_for_raise_or_promo', 'D44_b': 'received_raise_or_promo',
                'D44_c': 'applied_fore_new_job', 'D44_d': 'started_new_job',
                'D44_e': 'vol_left_job', 'D5': 'partner_work_for_pay',
                'D49': 'partner_work_more_35hrs', 'ppage': 'Age', 'ppgecat':'Age_category', # 1:18-24; 2:25-34; 3:35-44; 4:45-54; 5,6,7: 55 and over
                'ppeduc5': 'Education', 'ppgender': 'Gender', 
               }
work_col = work_col_map.keys()

In [ ]:
house_col_map = {'GH1': 'housing_type',  # {1: own hokme w mortgage, 2: own home clear, 3: rent, 4: neither own nor rent}
                 'R3': 'monthly_rent', 
                'M4': 'monthly_mortgage', 'pphhsize': 'Household_Size', 'ppkid017': 'Household_num_children',
                 'pphouse4':'Structure_Type', # {1: detached single family, 2: condo single family, 3: apartment building, 4: other (mobile home, boathouse, RV/van)}
                 'pprent': 'Ownership_Status', # {1: owned or buying, 2: rent for cash, 3: occupied without payment}
                }

house_col = house_col_map.keys()

In [ ]:
money_col_map = { 'I40': 'annual_income', 'I41_a': 'earned_income_tax_credit', 
                  'I41_b': 'SNAP', 'I41_c':'WIC', 'I41_d':'Housing_assistance', 
                  'I9': 'variable_income', # {1: roughly same, 2: varies a bit, 3: varies a lot}
                  'I20': 'total_spending', 'I21_a': 'total_income_vs_last_year',
                 'I21_b': 'total_spending_vs_last_year', 'INF4': 'financial_situation_vs_last_year',
                 'EF6C_a': 'pay_rent_or_mortgage_full', 'EF6C_a': 'pay_house_utilities_bills_full',
                 'atleast_okay': 'doing_ok_financially'
    
}
money_col = money_col_map.keys()

In [ ]:
shed[['GH1']].value_counts()

In [ ]:
shed[['pphouse4']].value_counts()

In [ ]:
shed[['pprent']].value_counts()

# Filter the data

In [ ]:
# ignore households who say thei occupy without paying
shed_copy = shed[shed['pprent'].str.contains('Owned|Rented')].copy()

In [ ]:
# ignore homeoweners who own house free and clear
shed_copy = shed_copy[shed_copy['GH1'].str.contains('home with a mortgage|Pay rent')].copy()

In [ ]:
shed_copy.shape

In [ ]:
# ignore non perment structure housing (mobile home, houseboat)
shed_copy = shed_copy[shed_copy['pphouse4'] != 'Other (mobile home, boat, RV, van, etc.)'].copy()
shed_copy.shape

In [ ]:
# focus on prime age households
# people not retired
# not full-time students
shed_copy = shed_copy[(shed_copy['ppage'] >24) & (shed_copy['ppage'] <55)].copy()
print(shed_copy.shape)
shed_copy = shed_copy[(shed_copy['D22_i'] !='Yes') ].copy()
print(shed_copy.shape)
shed_copy = shed_copy[(shed_copy['D22_h'] !='Yes') ].copy()
print(shed_copy.shape)

In [ ]:
# housing assisatnce, WIC, SNAP, housing assistance 'I41_d'
shed_copy = shed_copy[(shed_copy['I41_b'] =='No') & (shed_copy['I41_c'] =='No') &  (shed_copy['I41_d'] =='No')].copy()
shed_copy.shape

In [ ]:
# CLEAN the data. remove househodl who report payng less than $300 mortgage, less than $150 rent
# decioded by lookng at the hist of rent/mortgage, picking something reasonable


In [ ]:
shed_copy = shed_copy[(shed_copy['M4'] >20) | (shed_copy['R3'] >20) ].copy()
shed_copy.shape

In [ ]:
# remove teh top income bracket bc we don't have a mid-point
shed_copy = shed_copy[(shed_copy['I40'] !='$200,000 or more') ].copy()
shed_copy.shape

In [ ]:
# remove teh top income bracket bc we don't have a mid-point
shed_copy = shed_copy[(shed_copy['I40'] !='Less than $5,000') ].copy()
shed_copy.shape

In [ ]:
shed_copy['I40'].value_counts()

In [ ]:
# work for pay
shed_copy['D1A'].value_counts(dropna=False)

In [ ]:
shed_copy['works_for_pay'] = ((shed_copy['D1A']=='Yes')).astype(int)
shed_copy['Want_work_more'] = ( (shed_copy['D1E']=='Yes')).astype(int)

In [ ]:
shed_copy['works_more_35hrs'] = ( (shed_copy['D48']=='Yes') | (shed_copy['D3B']=='Yes')).astype(int)
shed_copy['works_multiple_jobs'] = ( (shed_copy['D4']=='Yes')).astype(int)
shed_copy['asked_for_raise'] = ( (shed_copy['D44_a']=='Yes')).astype(int)

In [ ]:
shed_copy['applied_or_started_new_job'] = ( (shed_copy['D44_c']=='Yes') | (shed_copy['D44_d']=='Yes')).astype(int)

In [ ]:
ax = shed_copy.R3.hist(bins=37, color='cadetblue')
plt.title('Histogram of Monthly Rents', size=13)
plt.xlabel('Dollars', size=12)
plt.ylabel('Count', size=12)
ax.tick_params(axis='both', which='major', labelsize=11)

plt.axvline(shed_copy.R3.median(), color='k', label='median')
plt.axvline(shed_copy.R3.mean(), color='k', ls='--', label='mean')

plt.axvline(shed_copy.R3.quantile(.25), color='b', label='middle 50%')
plt.axvline(shed_copy.R3.quantile(.75), color='b')
plt.legend(fontsize=12)

In [ ]:
shed_copy.R3.describe()

In [ ]:
shed_copy.R3.value_counts(dropna=False)

In [ ]:
shed_copy.M4.describe()


In [ ]:
ax = shed_copy.M4.hist(bins=45, color='mediumpurple')
plt.title('Histogram of Monthly Mortgages', size=13)
plt.xlabel('Dollars', size=12)
plt.ylabel('Count', size=12)
ax.tick_params(axis='both', which='major', labelsize=11)

plt.axvline(shed_copy.M4.median(), color='k', label='median')
plt.axvline(shed_copy.M4.mean(), color='k', ls='--', label='mean')

plt.axvline(shed_copy.M4.quantile(.25), color='b', label='middle 50%')
plt.axvline(shed_copy.M4.quantile(.75), color='b')
plt.legend(fontsize=12)

In [ ]:
income =shed_copy.I40.copy()

shed['income'] = income.str.replace('$', '')
shed['income'].value_counts()

In [ ]:
dollar_order = ['5,000 to 9,999', '10,000 to 14,999', '15,000 to 19,999', 
               '20,000 to 24,999', '25,000 to 29,999', '30,000 to 34,999', '35,000 to 39,999',
               '40,000 to 49,999', '50,000 to 59,999', '60,000 to 74,999', '75,000 to 99,999',
               '100,000 to 149,999', '150,000 to 199,999']#, '200,000 or more']


In [ ]:
shed['income'][~shed['R3'].isna()].value_counts()[dollar_order].plot.bar(color='cadetblue')
plt.title('Barplot of Income Brackets for Renters', size=13)
plt.xlabel('Dollars', size=12)
plt.ylabel('Count', size=12)
ax.tick_params(axis='both', which='major', labelsize=11)

In [ ]:
shed['income'][~shed['M4'].isna()].value_counts()[dollar_order].plot.bar(color='mediumpurple')
plt.title('Barplot of Income Brackets for Homeowners', size=13)
plt.xlabel('Dollars', size=12)
plt.ylabel('Count', size=12)
ax.tick_params(axis='both', which='major', labelsize=11)

In [ ]:
def get_midpoint(row):
    
    points = row.split(' to ')

    if points[0] == '200,000 or more':
        return 250000/12

    elif points[0] == 'Less than 5,000':
        return 2500/12
        
    else: 
        return (1+int(points[1].replace(',','').replace('$','')) + int(points[0].replace(',','').replace('$','')) )/24.

In [ ]:
monthly_income_midpoint = income.apply(get_midpoint)
shed_copy['monthly_income_midpoint'] = shed_copy.I40.str.replace('$', '').apply(get_midpoint)

In [ ]:
monthly_rent_to_income = 100*(shed_copy.R3/monthly_income_midpoint)

In [ ]:
monthly_mort_to_income = 100*(shed_copy.M4/monthly_income_midpoint)

In [ ]:
monthly_mort_to_income.describe()

In [ ]:
shed_copy['monthly_mort_to_income'] = monthly_mort_to_income
shed_copy['monthly_rent_to_income'] = monthly_rent_to_income


In [ ]:
monthly_rent_to_income.describe()

In [ ]:
monthly_rent_to_income.describe()

In [ ]:
ax = monthly_mort_to_income.round(0).clip(upper=100).hist(bins=51, alpha=.8, color='b', density=True, label='mortgage')
monthly_rent_to_income.round(0).clip(upper=100).hist(ax=ax,bins=51, alpha=.7, color='r', density=True, label='rent')
plt.legend(loc='upper center', fontsize=11)
plt.title('Mortgage/Rent Expenditure to Income Ratio')
plt.ylabel('density')
plt.xlabel('ratio (%)')

In [ ]:
# filter out hosueholds who spend more then 70% income on housing
shed_copy = shed_copy[(shed_copy['monthly_rent_to_income'] <80) | (shed_copy['monthly_mort_to_income'] <80)].copy()
shed_copy.shape

In [ ]:
shed_copy['monthly_rent_to_income'].describe()

In [ ]:
shed_copy['monthly_mort_to_income'].describe()

In [ ]:
ax = shed_copy['monthly_mort_to_income'].round(0).hist(bins=33, alpha=.8, color='mediumpurple', density=True, label='mortgage')
shed_copy['monthly_rent_to_income'].round(0).hist(ax=ax,bins=31, alpha=.7, color='cadetblue', density=True, label='rent')
plt.legend(loc='upper center', fontsize=12)
plt.title('Mortgage/Rent Expenditure to Income Ratio')
plt.ylabel('density', size=12)
plt.xlabel('ratio (%)', size=12)
ax.tick_params(axis='both', which='major', labelsize=11)

In [ ]:
shed_copy[shed_copy['GH1']!='Pay rent'][['monthly_income_midpoint', 'monthly_mort_to_income'] ].corr()

In [ ]:
plt.scatter( 
            shed_copy[shed_copy['GH1']!='Pay rent']['monthly_income_midpoint'] , 
shed_copy[shed_copy['GH1']!='Pay rent']['monthly_mort_to_income'],
color='k', marker='.' , alpha=.7, s=40)
plt.xlabel('Monthly Income (mid-point of reported income bracket)', size=11)
plt.ylabel('Housing Affordability IRM', size=11)
plt.title('Simple plot of income vs IRM (Homeowners)', size=12)

In [ ]:
work_model = logit("Want_work_more ~  monthly_mort_to_income",# + ppgender + ppkid017 + ppkid017*ppgender", 
                   data = shed_copy[shed_copy['GH1']=='Own your home with a mortgage or loan']) 
result = work_model.fit()
result.summary()

In [ ]:
work_model2 = logit("Want_work_more ~  monthly_mort_to_income + ppgender + ppkid017 + ppkid017*ppgender + ppmsacat +ppeduc5", 
                   data = shed_copy[shed_copy['GH1']=='Own your home with a mortgage or loan']) 
result2 = work_model2.fit()
result2.summary()

In [ ]:
ax = shed_copy[shed_copy['GH1']!='Pay rent'].plot.scatter(y='Want_work_more', x= 'monthly_mort_to_income', label='raw data')
plt.scatter( shed_copy[shed_copy['GH1']!='Pay rent']['monthly_mort_to_income'], result.predict() , color='r', marker='x', label='fitted values'  )
plt.title('Mortgage to Income Ratio vs Want Work More (no/yes)')
plt.ylabel('Want Work More', size=12)
plt.xlabel('Ratio (%)', size=12)
ax.tick_params(axis='both', which='major', labelsize=11)
plt.legend(fontsize=12)

In [ ]:
work_model = logit("Want_work_more ~  monthly_rent_to_income",# + ppgender + ppkid017 + ppkid017*ppgender", 
                   data = shed_copy[shed_copy['GH1']=='Pay rent']) 
result = work_model.fit()
result.summary()

In [ ]:
work_model2 = logit("Want_work_more ~  monthly_rent_to_income + ppgender + ppkid017 + ppkid017*ppgender + ppmsacat +ppeduc5", 
                   data = shed_copy[shed_copy['GH1']=='Pay rent']) 
result2 = work_model2.fit()
result2.summary()

In [ ]:
ax = shed_copy[shed_copy['GH1']=='Pay rent'].plot.scatter(y='Want_work_more', x= 'monthly_rent_to_income', label='raw data')
plt.scatter( shed_copy[shed_copy['GH1']=='Pay rent']['monthly_rent_to_income'], result.predict() , color='r', marker='x', label='fitted values'  )
#plt.scatter( shed_copy[shed_copy['GH1']=='Pay rent']['monthly_rent_to_income'], result2.predict() , color='m', marker='.', label='fitted values (full model)'  )

plt.title('Rent to Income Ratio vs Want Work More (no=0; yes=1)')
plt.ylabel('Want Work More', size=12)
plt.xlabel('Ratio (%)', size=12)
ax.tick_params(axis='both', which='major', labelsize=11)
plt.legend(fontsize=12)

In [ ]:
work_model = logit("works_for_pay ~  monthly_mort_to_income", 
                   data = shed_copy[shed_copy['GH1']!='Pay rent']) 
result = work_model.fit()
result.summary()

In [ ]:
work_model = logit("works_more_35hrs ~  monthly_mort_to_income", 
                   data = shed_copy[shed_copy['GH1']!='Pay rent']) 
result = work_model.fit()
result.summary()

In [ ]:
work_model = logit("works_multiple_jobs ~  monthly_mort_to_income", 
                   data = shed_copy[shed_copy['GH1']!='Pay rent']) 
result = work_model.fit()
result.summary()

In [ ]:
work_model = logit("asked_for_raise ~  monthly_mort_to_income", 
                   data = shed_copy[shed_copy['GH1']!='Pay rent']) 
result = work_model.fit()
result.summary()

In [ ]:
work_model = logit("applied_or_started_new_job ~  monthly_mort_to_income", 
                   data = shed_copy[shed_copy['GH1']!='Pay rent']) 
result = work_model.fit()
result.summary()